In [179]:
import requests
import sqlite3 as sql
import json

In [180]:
db=sql.connect('marketcap.db')
cur=db.cursor()

In [181]:
#For making sql table
# cur.execute("""CREATE TABLE market(
#                un_id INTEGER NOT NULL PRIMARY KEY UNIQUE,
#                link_id INTEGER,
#                date_time timestamp,
#                date text,
#                time text,
#                btc real,
#                open real,
#                volume real,
#                marketcap real
#                );
               
#                     """)

## you can change the range I mention in comments below to get the ID data from 1 to 100
## I only take Id from 1 to 4

In [245]:
cur.execute('select * from market limit 1')
sql_table=cur.fetchone()
if sql_table is None:
    un_id=0
    for var in range(1,5):   # change the range from 1 to 100     
        try:
            data=requests.get(f'https://web-api.coinmarketcap.com/v1.1/cryptocurrency/quotes/historical?convert=USD,BTC&format=chart_crypto_details&interval=1h&time_end=1598092056&time_start=1595413656&id={var}')
            ac_data=json.loads(data.text)
            link_id=var
            
            for val in ac_data['data'].items():
                ori_date_time=val[0]
                date_time=val[0].split('T')
                conv_data=val[1]
                un_id= un_id+1
                cur.execute("""INSERT INTO market(un_id,link_id,date_time,date,time,btc,open,volume,marketcap) 
                            VALUES(?,?,?,?,?,?,?,?,?)""",(un_id,link_id,ori_date_time,date_time[0],date_time[1],conv_data['BTC'][0],conv_data['USD'][0],conv_data['USD'][1],conv_data['USD'][2]))

                db.commit()
        except Exception as e:
            print(f"1. {e}")
            pass

else:
    cur.execute('select link_id from market')
    id_db=cur.fetchall()
    id_db_list=[a[0] for a in sorted(set(id_db))]
    total_id=[a for a in range(1,5)] #change range from 1 to 100
    if len(id_db_list)==len(total_id):
        cur.execute('select un_id from market ORDER BY un_id DESC LIMIT 1')
        un_id=cur.fetchone()
        un_id=un_id[0]
        for var in total_id:
            try:
                link_id=var
                cur.execute(f'SELECT date_time from market WHERE link_id={var} ORDER BY un_id DESC LIMIT 1')
                last_time=cur.fetchone()
                data=requests.get(f'https://web-api.coinmarketcap.com/v1.1/cryptocurrency/quotes/historical?convert=USD,BTC&format=chart_crypto_details&interval=1h&time_end=1598092056&time_start=1595413656&id={var}')
                ac_data=json.loads(data.text)
                time_data=list(ac_data['data'].keys())
                if last_time[0] in time_data:
                    index=time_data.index(last_time[0])
                if index-len(ac_data['data'].keys())==0:
                    pass
                else:
                    for val in time_data[index+1:]:
                        ori_date_time=val
                        date_time=val.split('T')
                        conv_data=ac_data['data'][val]
                        un_id= un_id+1
                        cur.execute("""INSERT INTO market(un_id,link_id,date_time,date,time,btc,open,volume,marketcap) 
                                 VALUES(?,?,?,?,?,?,?,?,?)""",(un_id,link_id,ori_date_time,date_time[0],date_time[1],conv_data['BTC'][0],conv_data['USD'][0],conv_data['USD'][1],conv_data['USD'][2]))
                        db.commit()
            except Exception as e:
                print(f"2. {e}")
                pass
        
    else:
        for var in id_db_list:
            try:
                link_id=var
                cur.execute(f'SELECT date_time from market WHERE link_id={var} ORDER BY un_id DESC LIMIT 1')
                last_time=cur.fetchone()
                cur.execute('select un_id from market ORDER BY un_id DESC LIMIT 1')
                un_id=cur.fetchone()
                un_id=un_id[0]
                data=requests.get(f'https://web-api.coinmarketcap.com/v1.1/cryptocurrency/quotes/historical?convert=USD,BTC&format=chart_crypto_details&interval=1h&time_end=1598092056&time_start=1595413656&id={var}')
                ac_data=json.loads(data.text)
                time_data=list(ac_data['data'].keys())
                if last_time[0] in time_data:
                    index=time_data.index(last_time[0])
                if index-len(ac_data['data'].keys())==0:
                    print('pass')
                    pass
                else:
                    for val in time_data[index+1:]:
                        ori_date_time=val
                        date_time=val.split('T')
                        conv_data=ac_data['data'][val]
                        un_id= un_id+1
                        cur.execute("""INSERT INTO market(un_id,link_id,date_time,date,time,btc,open,volume,marketcap) 
                                 VALUES(?,?,?,?,?,?,?,?,?)""",(un_id,link_id,ori_date_time,date_time[0],date_time[1],conv_data['BTC'][0],conv_data['USD'][0],conv_data['USD'][1],conv_data['USD'][2]))
                        db.commit()
            except Exception as e:
                print(f"3. {e}")
                pass
        for var in total_id:
            cur.execute('select un_id from market ORDER BY un_id DESC LIMIT 1')
            un_id=cur.fetchone()
            un_id=un_id[0]
            try:
                if var in id_db_list:
                    pass
                else:
                    data=requests.get(f'https://web-api.coinmarketcap.com/v1.1/cryptocurrency/quotes/historical?convert=USD,BTC&format=chart_crypto_details&interval=1h&time_end=1598092056&time_start=1595413656&id={var}')
                    ac_data=json.loads(data.text)
                    link_id=var
                    for val in ac_data['data'].items():
                        ori_date_time=val[0]
                        date_time=val[0].split('T')
                        conv_data=val[1]
                        un_id= un_id+1
                        cur.execute("""INSERT INTO market(un_id,link_id,date_time,date,time,btc,open,volume,marketcap) 
                                    VALUES(?,?,?,?,?,?,?,?,?)""",(un_id,link_id,ori_date_time,date_time[0],date_time[1],conv_data['BTC'][0],conv_data['USD'][0],conv_data['USD'][1],conv_data['USD'][2]))

                        db.commit()

            except Exception as e:
                print(f"4. {e}")
                pass


